# WH-3151 - E - Axis - NonFunded - Equifax Update Processing

- submits the bulk update with the Equifax XML data  
- the creation of the update requests is different notebook
    - see: WH-3151 - D - Axis - NonFunded - Equifax Update Mapping

** NOT RUN **

- Done by ROB

In [1]:
import traceback
import requests
import random
import boto3
import json
import time
import sys
import os
import shutil

This is all run in **staging**

## Warm-up lambda before submitting records for processing  
Not sure why we still do this ;-)  

In [2]:
warmup_request_staging = {
  "messageType": "prepareAnalytics",
  "messageContext": {
    "clientID": "healthcheck-staging"
  },
  "messageID": "case0001-jira-0535-0000-201810051837",
  "messageTimestamp": "an iso timestamp",
  "messageVersion": "messageVersion",
  "producerName": "warmup-before-bulk",
  "producerVersion": "producer-version",
  "messageData": {
    "clientID": "healthcheck-staging",
    "requestID": "case0001-jira-0535-0000-201810051837"
  }
}

In [3]:
arn_name = "arn:aws:lambda:us-west-2:098827237479:function:data-warehouse-pipeline-analytics-staging-preparingAnalytics"
input_request = warmup_request_staging

In [4]:
SUCCESS_STATUS_CODES = [202, 200]
client = boto3.client('lambda')
res = client.invoke(FunctionName=arn_name,InvocationType='Event',Payload=json.dumps(input_request))

if res['StatusCode'] not in SUCCESS_STATUS_CODES:
        print(res['StatusCode'])

In [5]:
time.sleep(15)

## Upload from local  
Put sets of the JSON into "Groups" that can be processed in bulk  
Note: too many in a  group can cause a problem - the size of the equifax update becomes too large for kinesis
Typically, 4-6 thousand individual JSON can be in a "group"
The number of groups then depends on how many equifax updates are in each saved JSON file (see "WH-3151 - D - Axis - NonFunded - Equifax Update Mapping")


### If everthing is fine, save them directly to S3    
* Bucket: "trustsciencespark"  
* Client: "axisautofinance-wh"  

**NOTE** setting "groups" manually.  

In [7]:
work_folder = "D:\\WH-3151\\"
output_folder = work_folder + "output\\"

In [8]:
# set this to how many sets of 1000 for each processing group

# since for WH-3151, the total is only xxxxxxx, just set to 6 and all in 1 groups

group_size = 6
group_list = []

for filename in os.listdir(output_folder):
    if filename.endswith(".json"):
        json_number = int(filename.split("update")[1].split(".")[0])
        group_number = int(json_number / group_size)
        group_folder = output_folder+"group{}\\".format(group_number)
        group_list.append(group_folder)
        os.makedirs(os.path.dirname(group_folder), exist_ok=True)
        print(output_folder+filename, group_folder+filename)
        shutil.copyfile(output_folder+filename, group_folder+filename)

D:\WH-3148\output\axisautofinance_equifax_update0.json D:\WH-3148\output\group0\axisautofinance_equifax_update0.json
D:\WH-3148\output\axisautofinance_equifax_update1.json D:\WH-3148\output\group0\axisautofinance_equifax_update1.json
D:\WH-3148\output\axisautofinance_equifax_update2.json D:\WH-3148\output\group0\axisautofinance_equifax_update2.json
D:\WH-3148\output\axisautofinance_equifax_update3.json D:\WH-3148\output\group0\axisautofinance_equifax_update3.json
D:\WH-3148\output\axisautofinance_equifax_update4.json D:\WH-3148\output\group0\axisautofinance_equifax_update4.json
D:\WH-3148\output\axisautofinance_equifax_update5.json D:\WH-3148\output\group0\axisautofinance_equifax_update5.json


### Do the S3 Sync to get from local to S3

In [9]:
group_folders = []
for folder in group_list:
    if folder not in group_folders:
        group_folders.append(folder)
sorted(group_folders)

['D:\\WH-3148\\output\\group0\\']

In [10]:
# since only 1, doing manually
sync_command = "aws s3 sync D:\WH-3151\output\group0\ s3://trustsciencespark/axisautofinance-wh/WH-3151/equifaxUpdates/group0/"
os.system(sync_command)

0

See:  
https://s3.console.aws.amazon.com/s3/buckets/trustsciencespark/axisautofinance-wh/WH-3151/equifaxUpdates/?region=us-west-2&tab=overview

## Bulk Processing  

- Send each group to the bulk update lambda
- make sure path is set to above
- the "updateRequst" is True   

In [13]:
# looped through groups **manually**
group_num = 0
bulk_request = {
  "requestPath": "trustsciencespark/axisautofinance-wh/WH-3151/equifaxUpdates/group{0}/".format(group_num),
  "clientId": "axisautofinance-wh",
  "userId": "rob",
  "priority": 1,
  "priorityOverwrite": False,
  "updateRequest": True,
  "source": "WH-3151-update".format(group_num)   # I don't think the bulkRequestProducer is using this in app_data_log
} 

print(bulk_request)

{'requestPath': 'trustsciencespark/axisautofinance-wh/WH-3148/equifaxUpdates/group0/', 'clientId': 'axisautofinance-wh', 'userId': 'rob', 'priority': 1, 'priorityOverwrite': False, 'updateRequest': True, 'source': 'WH-3148-update'}


In [14]:
# application-pipeline-staging-bulkRequestProducer
bulk_api_staging = "https://0h4pk200f0.execute-api.us-west-2.amazonaws.com/staging/api/bulk/request"

In [15]:
# Send bulk test
res = requests.post(bulk_api_staging, json=bulk_request)
print(res.status_code)
print(res.text)

202
{"status": "Accepted", "message": "The request was submitted successfully."}


### Output
Expected: 202  
{"status": "Accepted", "message": "The request was submitted successfully."}

Errors:  
If you get a 504 {"message": "Endpoint request timed out"} when trying to do 10x1000 size groups.  -  neeed many more groups.
Reorganize s3 - recopy from local to to S3

In [16]:
print("Submitted Request is: ")
print(bulk_request)

Submitted Request is: 
{'requestPath': 'trustsciencespark/axisautofinance-wh/WH-3148/equifaxUpdates/group0/', 'clientId': 'axisautofinance-wh', 'userId': 'rob', 'priority': 1, 'priorityOverwrite': False, 'updateRequest': True, 'source': 'WH-3148-update'}


## Done!
Checked each group in SQL.
Only 1 group and all 5398 were updated!

Example:  
- https://console.aws.amazon.com/s3/buckets/trustscience-test/axisautofinance-wh/00172a86-b027-476c-b312-ebb27a40a77d/?region=us-west-2&tab=overview 

#### Move on the loading the RDE ....